In [1]:
from keras.models import Sequential, load_model
from keras.layers import Dense, Flatten, Convolution2D, MaxPooling2D,Dropout, Reshape, LSTM, BatchNormalization,TimeDistributed
from keras.layers import Conv2D
from keras.optimizers import Adam #, RMSprop, SGD
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from time import time

import pickle
import random
import numpy as np
import sys
import h5py
import glob
import os

Using TensorFlow backend.


In [2]:
batch_size =  16 #multiple of 1290/5 (recording length)
nb_classes = 6
rows, cols = 431, 128   
nb_epoch = 200
pool_size = (5,5)                  # size of pooling area for max pooling
prob_drop_conv = 0.3                # drop probability for dropout @ conv layer
prob_drop_hidden = 0.3              # drop probability for dropout @ fc layer

In [3]:
#might need to rewrite these



def fileLists():
    trainlist=[]
    validationlist=[]
    testlist=[]
    evalSetupFiles='..\\..\\CASAdatasets\\DCASE18_ASCT1\\TUT-urban-acoustic-scenes-2018-development\\evaluation_setup\\*.txt'
    txtfilelist=glob.glob(evalSetupFiles)
    for txt in txtfilelist:
        if '_location' not in txt:
            continue
        with open(txt,'r') as evaltxtfile:
            for line in evaltxtfile.readlines():
                line=line.strip().split('\t')[0]
                if 'train' in txt:
                    trainlist.append(line.replace('audio','logMelSpec').replace('.wav','_aggScenes.pckl').replace('/','\\'))
                elif 'test' in txt:
                    testlist.append(line.replace('audio','logMelSpec').replace('.wav','_aggScenes.pckl').replace('/','\\'))
                else:
                    validationlist.append(line.replace('audio','logMelSpec').replace('.wav','_aggScenes.pckl').replace('/','\\'))
    print('trainfiles: ', str(len(trainlist)))
    print('validationfiles: ', str(len(validationlist)))
    print('testfiles: ', str(len(testlist)))
    return trainlist,validationlist,testlist

labelRef={'barcelona':0, 'helsinki':1, 'london':2, 'paris':3, 'stockholm':4, 'vienna':5}

def getData(flist):
    pth='..\\..\\CASAdatasets\\DCASE18_ASCT1\\TUT-urban-acoustic-scenes-2018-development\\'
    X_=np.zeros(((len(flist)),128,431))
    Y_=np.zeros(((len(flist)),nb_classes))
    for i,tfile in enumerate(flist):
        with open(pth+tfile, "rb" ) as scenesample:
            fv=pickle.load(scenesample)
        X_[i,:,:]=fv
        location=tfile.split('-')[1]
        Y_[i,labelRef[location]]=1
    return X_, Y_
#trainlist,validationlist,testlist=fileLists()
#X_train, Y_train = getData(trainlist)
#X_val, Y_val = getData(validationlist)
#X_test, Y_test = getData(testlist)

print(' data sorted')


 data sorted


In [6]:
###### Convolutional model
def compileCRNN(cols,rows,nb_classes=1):

    model = Sequential()
    # conv1 layer
    model.add(Conv2D(32, (7, 7), padding='same', activation='relu', input_shape=(cols,rows,1)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=pool_size, strides=(2), padding='same'))
    model.add(Dropout(prob_drop_conv))

    # conv2 layer
    model.add(Conv2D(64, (7, 7), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(4,7), strides=(2), padding='same'))
    model.add(Dropout(prob_drop_conv))

    # conv3 layer
    model.add(Conv2D(128, (2, 2), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=pool_size, strides=(2), padding='same'))
    model.add(BatchNormalization())
    model.add(Dropout(prob_drop_conv))

    model.add(Flatten())
    # fc1 layer
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(prob_drop_hidden))
    model.add(BatchNormalization())
   
    # fc2 layer
    model.add(Dense(nb_classes, activation='softmax'))
    opt = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False) 
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()
    return model

model = compileCRNN(cols,rows,nb_classes=nb_classes)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 128, 431, 32)      1600      
_________________________________________________________________
batch_normalization_7 (Batch (None, 128, 431, 32)      128       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 64, 216, 32)       0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 64, 216, 32)       0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 64, 216, 64)       100416    
_________________________________________________________________
batch_normalization_8 (Batch (None, 64, 216, 64)       256       
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 32, 108, 64)       0         
__________

In [7]:
# Train
#fold=1
def train_network(model, model_name, X_train, Y_train, X_val, Y_val, nb_epoch, validationsplit_size, batchsize, early_stoping_patience, output_folder):
    checkpointer = ModelCheckpoint(filepath=os.path.join(output_folder,model_name + '.hdf5'),save_best_only=True)
    early_stopping = EarlyStopping(monitor='val_loss', patience=early_stoping_patience)
    tensorboard = TensorBoard(log_dir="logs\\{}".format(time()))
    Callbacks=[checkpointer,  tensorboard] #early_stopping,
  #  print(samples)
    steps=int(samples/batchsize)
    validationsteps=int(validationsplit_size/batchsize)
    history = model.fit(X_train, Y_train, epochs=nb_epoch, callbacks=Callbacks, batch_size=batch_size, validation_data=(X_val, Y_val), shuffle=True, verbose=1)
    return history,model

def buildModel(savemodelfilename, samples,model,X_train, Y_train,X_val, Y_val):
    valSplit_size = int(samples/4)
    early_stoping_patience=5
    history,model = train_network(model, 'models\\best_model_simple', X_train, Y_train, X_val, Y_val, nb_epoch, valSplit_size, batch_size, early_stoping_patience,'.')
    model.save_weights(savemodelfilename)
    return model, history


In [11]:
from datetime import datetime
batch_size =  16 #multiple of 1290/5 (recording length)
nb_classes = 6
rows, cols = 431, 128  

trainlist,validationlist,testlist=fileLists()
print('lists sorted')
X_tr, Y_train = getData(trainlist)
X_v, Y_val = getData(validationlist)
X_te, Y_test = getData(testlist)
print('data obtained')
X_train=np.expand_dims(X_tr,axis=3)
X_val=np.expand_dims(X_v,axis=3)
X_test=np.expand_dims(X_te,axis=3)
print(X_train.shape, X_val.shape, X_test.shape)
print('data sorted')
model = compileCRNN(cols,rows,nb_classes=nb_classes)
print('model compiled')
savemodelfilename='models\\final_model_simple.testsave'
samples=(X_train.shape[0])
print(datetime.now())
models,histories= buildModel(savemodelfilename, samples,model,X_train,Y_train,X_val, Y_val)
print(datetime.now())
print('model saved')

trainfiles:  6122
validationfiles:  2518
testfiles:  2518
lists sorted
data obtained
(6122, 128, 431, 1) (2518, 128, 431, 1) (2518, 128, 431, 1)
data sorted
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_16 (Conv2D)           (None, 128, 431, 32)      1600      
_________________________________________________________________
batch_normalization_19 (Batc (None, 128, 431, 32)      128       
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 64, 216, 32)       0         
_________________________________________________________________
dropout_19 (Dropout)         (None, 64, 216, 32)       0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 64, 216, 64)       100416    
_________________________________________________________________
batch_normalization_20 (Batc (None, 64, 216, 64)   

Epoch 100/200
6122/6122 [==============================] - 25s 4ms/step - loss: 0.2030 - acc: 0.9325 - val_loss: 7.4399 - val_acc: 0.3646
Epoch 101/200
6122/6122 [==============================] - 25s 4ms/step - loss: 0.2161 - acc: 0.9268 - val_loss: 5.2097 - val_acc: 0.4786
Epoch 102/200
6122/6122 [==============================] - 25s 4ms/step - loss: 0.1885 - acc: 0.9342 - val_loss: 7.5809 - val_acc: 0.3320
Epoch 103/200
6122/6122 [==============================] - 25s 4ms/step - loss: 0.2671 - acc: 0.9090 - val_loss: 3.3762 - val_acc: 0.4746
Epoch 104/200
6122/6122 [==============================] - 24s 4ms/step - loss: 0.2294 - acc: 0.9187 - val_loss: 4.7131 - val_acc: 0.4063
Epoch 105/200
6122/6122 [==============================] - 24s 4ms/step - loss: 0.1903 - acc: 0.9352 - val_loss: 3.3515 - val_acc: 0.5234
Epoch 106/200
6122/6122 [==============================] - 24s 4ms/step - loss: 0.1611 - acc: 0.9499 - val_loss: 6.3738 - val_acc: 0.3368
Epoch 107/200
6122/6122 [=========